In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/데이터전처리

/content/drive/MyDrive/데이터전처리


In [3]:
!pip install transformers

In [28]:
import pandas as pd
import random
import pprint # 보기 쉽게 깔끔한 형태로 출력

from transformers import AutoModel, AutoTokenizer

from torch import nn # 신경망(neural network)생성
import torch.nn.functional as F # 활성화함수, 손실함수를 대체하여 코드를 간결하게 만듬

from sklearn.metrics import accuracy_score, f1_score

In [29]:
data = pd.read_excel("./data/data_re_all_label.xlsx", engine="openpyxl")
display(data.head(2))
data.shape

,SENTENCE,label_idx
0,지금 배달되나요?,"487,3103,265"
1,아 네 배달됩니다,265


(15726, 2)

In [30]:
all_indices = data.index

In [31]:
train_size = int(0.8 * len(all_indices)) # data의 70% 훈련용
val_size = len(all_indices) - train_size # data의 30% 검증용
train_idx, val_idx = all_indices[:int(0.8 * len(all_indices))], all_indices[int(0.8 * len(all_indices)):]

num_classes = int(max(data["label_idx"].tolist())) + 1

In [32]:
train_idx, val_idx

(RangeIndex(start=0, stop=12580, step=1),
 RangeIndex(start=12580, stop=15726, step=1))

In [33]:
import torch
from torch.utils.data import DataLoader, Dataset, random_split
from transformers import BertForSequenceClassification, BertTokenizer, AdamW, BertModel
from transformers import get_constant_schedule_with_warmup, get_linear_schedule_with_warmup # 학습 스케줄러를 생성
from tqdm.auto import tqdm # 프로그램 실행 진행사항 확인

# transformers 라이브러리에서 BERT 토크나이저를 임포트합니다.
from transformers import BertTokenizer

# 다국어 지원 BERT 모델을 사용하여 토크나이저 객체를 생성합니다.
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

tokenizer_config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/425 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

- BERT 입력형태
  - [CLS] 토큰 + 첫번째 문장 + [SEP] 토큰 + 두번째 문장
    - 문장의 시작과 끝, 문장 간의 구분을 명확하게 인식 가능
  - [CLS] 토큰
    - "Classification token", 입력 문장의 시작 부분에 추가.
    - 이 토큰의 임베딩은 문장 분류 작업에 사용, 문장 전체의 문맥을 요약하는 역할
  - [SEP] 토큰
    - "Separator token", 두 문장을 구분하는 데 사용
  - 임베딩
    - 텍스트 데이터를 컴퓨터가 이해하고 처리할 수 있는 수치형 벡터로 변환하는 과정 혹은 변환된 벡터


In [34]:
class TextClassificationDatasetBatch(Dataset):
    def __init__(self, df, tokenizer, batch_size=4):
        self.tokenizer = tokenizer # 토크나이저를 클래스 변수로 저장
        self.sentences = df["SENTENCE"].tolist() # "sentence" 열의 데이터를 리스트로 변환하여 저장
        self.labels = df["label_idx"].tolist() # "label_idx" 열의 데이터를 리스트로 변환하여 저장

    def __len__(self): # 데이터 셋의 총 길이를 반환(레이블 개수 반환)
        return len(self.labels)

    def __getitem__(self, idx):
        # to_return = {
        #     "sentence_encoded":self.encodings[idx],
        #     "label":self.labels[idx]
        # }
        return (self.sentences[idx], self.labels[idx]) # 인덱스에 해당하는 문장과 레이블을 튜플로 묶어 반환

In [35]:
train_data = data.iloc[train_idx]
val_data = data.iloc[val_idx]

In [36]:
# train_dataset = TextClassificationDataset(train_data, tokenizer)
# val_dataset = TextClassificationDataset(val_data, tokenizer)
train_dataset = TextClassificationDatasetBatch(train_data, tokenizer)
val_dataset = TextClassificationDatasetBatch(val_data, tokenizer)

In [37]:
train_dataset[0]

('지금 배달되나요?', '487,3103,265')

In [ ]:
# DataLoader : PyTorch에서 데이터 로드하는데 사용하는 클래스
# 데이터셋을 batch_size만큼 분할하여 로드
# shuffle=True: epoch마다 데이터셋을 무작위로 섞음(과적합 방지)
# shuffle=False: 데이터셋을 섞지않음(검증 데이터는 순서가 결과에 영향을 미치지 않기때문)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [ ]:
# 'next', 'iter' 함수를 이용해 DataLoader 객체에서 첫 번째 배치를 가져옴
# next: 반복자의 다음 값을 반환, 모든 값 소진시 'StopIteration'예외 발생
# iter: 객체를 인자로 받아 해당 객체의 반복자(iterator)를 반환
t = next(iter(train_dataloader))

In [ ]:
t[0]

{'input_ids': tensor([[    0,  3729,  2223,  9893,  2210, 28078, 15302,  2170,  7937,  2088,
          3803,  5110, 16809,  2182,     2,     1,     1,     1,     1,     1],
        [    0, 12131,  2223,  5971,     2,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1],
        [    0,  1439, 25741,  2052,  2182,     2,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1],
        [    0,   752,     2,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1],
        [    0,  4750,  2170,  2259,  5447,  3776,  2062,  3978,  5110,  2227,
          2182,    35,     2,     1,     1,     1,     1,     1,     1,     1],
        [    0,  3935, 11187,  5624,  2259, 11905,  2075,  1882,  2087, 11962,
         19572,  2292,  3041,  3667,  1326,  6233,  5624,  2182,     2,     1],
        [    0,  1491,  2019,  2

In [ ]:
t[1]

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

- input_ids: 전체 입력 데이터의 token id
- attention_mask: 연산이 필요없는 패딩 토큰에 대해 어텐션을 하지 않도록 마스킹해주는 역할
- token_type_ids: 각 token이 어떤 타입에 속하는지 나타내는 역할

In [ ]:
for i, t_data in enumerate(train_dataloader):
  tokenized = tokenizer(t_data[0], return_tensors="pt", padding=True, truncation=True, max_length=512)
  print(tokenized["input_ids"].shape, model(**tokenized)["pooler_output"].shape)
  # model(**tokenized) == model(input_ids=tokenized["input_ids"], token_type_ids=tokenized["token_type_ids"], attention_mask=tokenized["attention_mask"])

In [ ]:
# tokenized.keys()

In [ ]:
# model(**tokenized)

In [38]:
import torch
import torch.nn as nn


# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
num_classes = int(max(data["label_idx"].tolist())) + 1
epochs = 100  # 에폭 수
batch_size = 32  # 배치 크기
learning_rate = 1e-4  # 학습률

In [39]:
from sklearn.preprocessing import MultiLabelBinarizer
from functools import partial
import numpy as np

def collate_bert_tokenizer(samples, mlb):
    sentences = [s[0] for s in samples]
    labels = [s[1].split(',') for s in samples]  # 레이블을 쉼표로 분리합니다.

    # 레이블을 정수로 변환
    labels = mlb.transform(labels)

    global tokenizer

    # 데이터 확인
    # print("Before Tokenization: ", sentences)
    # tokenized_sentences = tokenizer(sentences, return_tensors='pt', padding=True, truncation=True, max_length=512)
    # print("After Tokenization: ", tokenized_sentences)

    # print("Before Tokenization: ", labels)
    # tokenized_labels = torch.tensor(labels, dtype=torch.float)
    # print("After Tokenization: ", tokenized_labels)

    return tokenizer(sentences, return_tensors='pt', padding=True, truncation=True, max_length=512), torch.tensor(labels, dtype=torch.float)  # 레이블의 데이터 타입을 float로 설정합니다.

train_data_values = train_data.values
val_data_values = val_data.values

# 문장과 레이블로 분리
train_sentences, train_labels = train_data_values[:, 0], train_data_values[:, 1]
val_sentences, val_labels = val_data_values[:, 0], val_data_values[:, 1]

all_labels = [label.split(',') for label in np.concatenate([train_labels, val_labels])]  # 'train_labels'와 'val_labels'를 합친 리스트에서 레이블 추출
mlb = MultiLabelBinarizer()
mlb.fit(all_labels)

# DataLoader 생성
train_dataloader = DataLoader(list(zip(train_sentences, train_labels)), shuffle=True, batch_size=32, collate_fn=partial(collate_bert_tokenizer, mlb=mlb))
val_dataloader = DataLoader(list(zip(val_sentences, val_labels)), shuffle=False, batch_size=32, collate_fn=partial(collate_bert_tokenizer, mlb=mlb))


In [ ]:
# train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn = collate_bert_tokenizer)
# val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn = collate_bert_tokenizer)

In [ ]:
# 'train_data'와 'val_data'의 길이 출력
print(len(train_data))
print(len(val_data))

# 'train_dataloader'와 'val_dataloader'의 첫 번째 배치 출력
for batch in train_dataloader:
    print(batch)
    break

for batch in val_dataloader:
    print(batch)
    break

In [ ]:
# # nn.Module을 상속하여 사용자 정의 모델 클래스 생성
# class CustomModel(nn.Module):
#     def __init__(self, bert_model, num_classes):
#         super(CustomModel, self).__init__() # 부모 클래스의 생성자 호출
#         # 사전 학습된 BERT 모델을 불러옴
#         self.bert = BertModel.from_pretrained(bert_model)
#         # 드롭아웃 레이어를 추가(과적합 방지)
#         self.dropout = nn.Dropout(0.1)
#         # 출력 레이어 추가(BERT 모델의 hidden_size를 입력받고, 출력 크기는 분류할 클래스의 개수)
#         self.fc = nn.Linear(self.bert.config.hidden_size, num_classes)

#     def forward(self, input_ids, attention_mask):
#         # 입력 데이터를 BERT 모델에 통과
#         outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
#         # BERT 모델의 출력 중 pooler_output(문장의 전체적인 의미)을 사용
#         pooled_output = outputs.pooler_output
#         # 드롭아웃 레이어 통과
#         dropout_output = self.dropout(pooled_output)
#         # 출력 레이어를 통과시켜 최종 출력을 반환
#         return self.fc(dropout_output)

# # CustomModel 클래스를 사용하여 모델을 생성합니다. BERT 모델로는 "klue/roberta-large"을,
# # 분류할 클래스의 개수는 num_classes(3139)를 사용합니다.
# model = CustomModel("klue/roberta-small", num_classes).to(device)


# optimizer = AdamW(model.parameters(),lr=learning_rate)
# # 학습 스케줄러를 생성(학습률을 동적으로 조정)
# scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_dataloader) * epochs)

You are using a model of type roberta to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of BertModel were not initialized from the model checkpoint at klue/roberta-small and are newly initialized: ['encoder.layer.2.attention.self.value.bias', 'encoder.layer.1.attention.output.dense.bias', 'encoder.layer.4.attention.output.dense.bias', 'encoder.layer.4.intermediate.dense.weight', 'encoder.layer.0.intermediate.dense.weight', 'encoder.layer.3.attention.self.key.bias', 'encoder.layer.5.attention.output.dense.bias', 'encoder.layer.3.output.LayerNorm.weight', 'encoder.layer.2.output.LayerNorm.weight', 'encoder.layer.1.output.LayerNorm.weight', 'encoder.layer.4.attention.output.LayerNorm.bias', 'encoder.layer.3.output.dense.bias', 'encoder.layer.1.output.LayerNorm.bias', 'encoder.layer.0.output.dense.bias', 'encoder.layer.4.attention.self.key.weight', 'encoder.layer.1.attention.self.key.bias', 'encoder.layer.3.interm

In [40]:
import torch
from torch import nn

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class BiRNN(nn.Module):
    def __init__(self, bert_model, num_classes):
        super(BiRNN, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model)
        self.dropout = nn.Dropout(0.5)  # Increase dropout rate
        self.linears = nn.ModuleList([nn.Linear(self.bert.config.hidden_size, 1) for i in range(num_classes)])
        self.num_classes = num_classes

        # 가중치 초기화
        for linear in self.linears:
            nn.init.xavier_uniform_(linear.weight)
            nn.init.zeros_(linear.bias)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        dropout_output = self.dropout(pooled_output)

        results = []
        for each_class_label_idx in range(self.num_classes):
            results.append(
                self.linears[each_class_label_idx](dropout_output)
            )

        return torch.cat(results, dim=1)

# 모델 생성
num_classes = len(mlb.classes_)
model = BiRNN("klue/bert-base", num_classes).to(device)

optimizer = AdamW(model.parameters(),lr=learning_rate, weight_decay=0.01)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_dataloader) * epochs)

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
# class EarlyStopping:
#     """조기 종료를 위한 클래스"""
#     def __init__(self, patience=7, verbose=False, delta=0):
#         """
#         Args:
#             patience (int): 얼마나 오차가 증가해야 조기 종료할 것인지 결정하는 인자
#             verbose (bool): True인 경우 각 조기 종료 단계에서 메시지를 출력
#             delta (float): 개선되었다고 인정하기 위한 최소 변화량
#         """
#         self.patience = patience
#         self.verbose = verbose
#         self.counter = 0
#         self.best_score = None
#         self.early_stop = False
#         self.val_loss_min = np.Inf
#         self.delta = delta

#     def __call__(self, val_loss, model):

#         score = -val_loss

#         if self.best_score is None:
#             self.best_score = score
#             self.save_checkpoint(val_loss, model)
#         elif score < self.best_score + self.delta:
#             self.counter += 1
#             print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
#             if self.counter >= self.patience:
#                 self.early_stop = True
#         else:
#             self.best_score = score
#             self.save_checkpoint(val_loss, model)
#             self.counter = 0

#     def save_checkpoint(self, val_loss, model):
#         '''검증 손실이 감소하면 모델을 저장합니다.'''
#         if self.verbose:
#             print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
#         torch.save(model.state_dict(), 'checkpoint.pt')
#         self.val_loss_min = val_loss


In [41]:
loss_f = nn.MultiLabelSoftMarginLoss().to(device)

for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    print('-' * 10)
    # 모델을 학습 모드로 설정합니다.
    model.train()
    # 입력 데이터와 레이블 전처리
    total_loss = 0
    for i, (_input, _label) in enumerate(tqdm(train_dataloader, desc="Training")):
        input_ids = torch.tensor(_input['input_ids']).to(device)
        attention_mask = torch.tensor(_input['attention_mask']).to(device)
        _label = torch.tensor(_label).to(device)

        outputs = model(input_ids, attention_mask)
        loss = loss_f(outputs, _label)

        loss.backward() # 역전파를 수행합니다.
        optimizer.step() # optimizer를 사용하여 파라미터 업데이트
        optimizer.zero_grad() # 그라디언트 초기화
        scheduler.step()

        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_dataloader)
    print(f"Training Loss: {avg_train_loss}")

    # 모델을 평가 모드로 설정합니다.
    model.eval()
    all_predictions = []
    all_labels = []
    total_val_loss = 0

    for i, (_input, _label) in enumerate(tqdm(val_dataloader, desc="Validation")):
        input_ids = torch.tensor(_input['input_ids']).to(device)
        attention_mask = torch.tensor(_input['attention_mask']).to(device)
        _label = torch.tensor(_label).to(device)

        outputs = model(input_ids, attention_mask)
        loss = loss_f(outputs, _label)
        total_val_loss += loss.item()

        # 모델의 예측과 실제 레이블을 저장
        # 예측은 각 레이블에 대한 확률을 이진 벡터로 변환
        all_predictions.extend( (torch.sigmoid(outputs) > 0.5).tolist() )
        all_labels.extend(_label.tolist())

    avg_val_loss = total_val_loss / len(val_dataloader)
    print(f"Validation Loss: {avg_val_loss}")

    # 정확도와 가중치 평균 F1 score 계산하여 출력
    # 'average' 파라미터를 'samples'로 설정하여 각 샘플의 레이블 간 평균을 계산
    print(f"Accuracy: {accuracy_score(all_labels, all_predictions)}")
    print(f'F1_score: {f1_score(all_labels, all_predictions, average="samples")}')



Epoch 1/100
----------


Training:   0%|          | 0/394 [00:00<?, ?it/s]

<ipython-input-41-25fb1f708505>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.tensor(_input['input_ids']).to(device)
<ipython-input-41-25fb1f708505>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask = torch.tensor(_input['attention_mask']).to(device)
<ipython-input-41-25fb1f708505>:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  _label = torch.tensor(_label).to(device)


Training Loss: 0.07946650783748833


Validation:   0%|          | 0/99 [00:00<?, ?it/s]

<ipython-input-41-25fb1f708505>:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.tensor(_input['input_ids']).to(device)
<ipython-input-41-25fb1f708505>:36: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask = torch.tensor(_input['attention_mask']).to(device)
<ipython-input-41-25fb1f708505>:37: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  _label = torch.tensor(_label).to(device)


Validation Loss: 0.0037270651499957145
Accuracy: 0.0
F1_score: 0.0
Epoch 2/100
----------


Training:   0%|          | 0/394 [00:00<?, ?it/s]

<ipython-input-41-25fb1f708505>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.tensor(_input['input_ids']).to(device)
<ipython-input-41-25fb1f708505>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask = torch.tensor(_input['attention_mask']).to(device)
<ipython-input-41-25fb1f708505>:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  _label = torch.tensor(_label).to(device)


KeyboardInterrupt: ignored

In [ ]:
# 임의의 입력 생성
input_ids = torch.randint(0, model.bert.config.vocab_size, (1, model.bert.config.max_position_embeddings)).to(device)
attention_mask = torch.ones((1, model.bert.config.max_position_embeddings)).to(device)

# 모델의 출력 확인
outputs = model(input_ids, attention_mask)
print(outputs)


tensor([[ 0.7705,  0.5602, -0.9317,  ..., -0.4622,  0.3850, -1.4787]],
       device='cuda:0', grad_fn=<CatBackward0>)


In [ ]:
# 임의의 레이블 생성
_label = torch.randint(0, 2, (1, num_classes)).to(device)

# 손실 함수 계산
loss_f = nn.MultiLabelSoftMarginLoss().to(device)
loss = loss_f(outputs, _label)

print(loss)


tensor(0.7782, device='cuda:0', grad_fn=<MeanBackward0>)


In [ ]:
while True:
  # 임의의 문장을 입력합니다
  test_sentence = input()
  if test_sentence == '수고하세요':
    break

  # 문장을 토큰화합니다
  inputs = tokenizer(test_sentence, return_tensors="pt", padding=True, truncation=True, max_length=512)
  if 'token_type_ids' in inputs:
    del inputs['token_type_ids']

  # 모델을 평가 모드로 설정하고 GPU로 이동
  model.eval()
  inputs = {k: v.to(device) for k, v in inputs.items()}

  # 모델에서 예측 수행
  with torch.no_grad():
      outputs = model(**inputs)
      predictions = outputs.argmax(dim=1)
      predicted_label = predictions.cpu().numpy()[0]

  # 예측된 라벨 출력
  print(f"Input Sentence: {test_sentence}")
  print(f"Predicted Label: {predicted_label}")

안녕하세요
Input Sentence: 안녕하세요
Predicted Label: 0
치킨 주문하고싶은데요
Input Sentence: 치킨 주문하고싶은데요
Predicted Label: 618
네. 메뉴를 선택해주시겠어요
Input Sentence: 네. 메뉴를 선택해주시겠어요
Predicted Label: 998
음... 간장 후라이드 반반 세트로 하고싶어요
Input Sentence: 음... 간장 후라이드 반반 세트로 하고싶어요
Predicted Label: 2432
세트메뉴로 하고싶으신건가요?
Input Sentence: 세트메뉴로 하고싶으신건가요?
Predicted Label: 2448
네. 간장치킨 세트요
Input Sentence: 네. 간장치킨 세트요
Predicted Label: 2759
네 알겠습니다.
Input Sentence: 네 알겠습니다.
Predicted Label: 0
#주소# 가 어떻게 되시나요?
Input Sentence: #주소# 가 어떻게 되시나요?
Predicted Label: 1876
#주소#로 배달해주시면 됩니다.
Input Sentence: #주소#로 배달해주시면 됩니다.
Predicted Label: 1876
네 주문해주셔서 감사합니다.
Input Sentence: 네 주문해주셔서 감사합니다.
Predicted Label: 536
수고하세요
